In [9]:
import torch
import pickle
import pandas as pd

from utils.utils import create_data_pipeline
from models.mohamed_ashraf.bilstm import BiLSTM

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [11]:
with open('utils/diacritic2id.pickle', 'rb') as f:
    diacritic2idx = pickle.load(f)

with open('utils/letter2idx.pickle', 'rb') as f:
    letter2idx = pickle.load(f)

In [12]:
vocab_size = len(letter2idx)
num_classes = len(diacritic2idx)

print(f"Vocab size: {vocab_size}, Number of classes: {num_classes}")

Vocab size: 38, Number of classes: 16


In [13]:
def pad_collate_fn(batch):
    x_batch, y_batch, mask_batch = zip(*batch)
    lengths_x = [len(x) for x in x_batch]
    x_padded = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=letter2idx['<PAD>'])
    y_padded = torch.nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=diacritic2idx['<PAD>'])
    mask_spadded = torch.nn.utils.rnn.pad_sequence(mask_batch, batch_first=True, padding_value=0)
    return x_padded, y_padded, mask_spadded, torch.tensor(lengths_x, dtype=torch.long)

In [14]:
test_dataset,  test_loader= create_data_pipeline(
    corpus_path='sample_test_no_diacritics.txt', 
    letter2idx=letter2idx, 
    diacritic2idx=diacritic2idx, 
    collate_fn=pad_collate_fn,
    train=False, 
    batch_size=32
)

In [15]:
model = BiLSTM(vocab_size=vocab_size, num_classes=num_classes)

In [16]:
checkpoint = torch.load("./models/mohamed_ashraf/bilstm.pth", map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

# print(f"Loaded checkpoint from epoch {checkpoint['epoch']}")
# print(f"Validation Loss: {checkpoint['val_loss']:.4f}")
# print(f"Validation Accuracy: {checkpoint['val_accuracy']:.4f}")

/tmp/ipykernel_115781/1611503649.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("./models/mohamed_ashraf/bilstm.pth", map_location=device)


BiLSTM(
  (embedding): Embedding(38, 128, padding_idx=13)
  (bilstm): LSTM(128, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=16, bias=True)
)

In [17]:
def create_predictions_csv(model, test_loader, letter2idx, output_file='predictions.csv'):
    model.eval()
    space_idx = letter2idx.get(' ', None)
    pad_idx = letter2idx.get('<PAD>', None)
    
    results = []
    row_id = 0
    
    with torch.no_grad():
        for batch_X, _, _, lengths in test_loader:
            batch_X = batch_X.to(device)
            
            outputs = model(batch_X, lengths)
            preds = outputs.argmax(dim=-1)
            
            batch_X_cpu = batch_X.cpu().numpy()
            preds_cpu = preds.cpu().numpy()
            
            for i in range(batch_X.size(0)):
                seq_len = lengths[i].item()
                
                for j in range(seq_len):
                    letter_idx = int(batch_X_cpu[i, j])
                    diacritic_idx = int(preds_cpu[i, j])
                    
                    if letter_idx == pad_idx or letter_idx == space_idx:
                        continue
                    
                    results.append({
                        'ID': row_id,
                        'label': diacritic_idx
                    })
                    row_id += 1
    
    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)
    
    return df

In [18]:
predictions_df = create_predictions_csv(
    model=model,
    test_loader=test_loader,
    letter2idx=letter2idx,
    output_file='predictions.csv'
)

In [19]:
def calculate_accuracy(gold_csv_path, predictions_csv_path):
    gold_df = pd.read_csv(gold_csv_path)
    pred_df = pd.read_csv(predictions_csv_path)
    
    print(f"📂 Loaded files:")
    print(f"  Gold standard: {len(gold_df):,} rows")
    print(f"  Predictions: {len(pred_df):,} rows")
    print(f"  Gold columns: {gold_df.columns.tolist()}")
    print(f"  Pred columns: {pred_df.columns.tolist()}")
    
    if len(gold_df) != len(pred_df):
        print(f"\n⚠️  Warning: Row count mismatch!")
        print(f"  Gold: {len(gold_df):,} | Predictions: {len(pred_df):,}")
        min_len = min(len(gold_df), len(pred_df))
        print(f"  Comparing first {min_len:,} rows only")
        gold_df = gold_df.head(min_len)
        pred_df = pred_df.head(min_len)
    
    merged_df = pd.merge(gold_df, pred_df, on='ID', suffixes=('_gold', '_pred'))
    
    correct = (merged_df['label_gold'] == merged_df['label_pred']).sum()
    total = len(merged_df)
    accuracy = (correct / total) * 100
    
    print(f"\n" + "="*60)
    print(f"📊 Accuracy Results")
    print(f"="*60)
    print(f"  Total samples: {total:,}")
    print(f"  Correct predictions: {correct:,}")
    print(f"  Incorrect predictions: {total - correct:,}")
    print(f"  Accuracy: {accuracy:.4f}%")
    print(f"="*60)
    
    print(f"\n📈 Per-Label Analysis:")
    unique_labels = sorted(merged_df['label_gold'].unique())
    
    for label in unique_labels:
        label_mask = merged_df['label_gold'] == label
        label_total = label_mask.sum()
        label_correct = ((merged_df['label_gold'] == label) & 
                         (merged_df['label_pred'] == label)).sum()
        label_acc = (label_correct / label_total * 100) if label_total > 0 else 0
        print(f"  Label {label}: {label_correct:4d}/{label_total:4d} correct ({label_acc:6.2f}%)")
    
    return accuracy, merged_df

In [20]:
accuracy, comparison_df = calculate_accuracy(
    gold_csv_path='sample_test_set_gold.csv',
    predictions_csv_path='predictions.csv'
)

📂 Loaded files:
  Gold standard: 182 rows
  Predictions: 182 rows
  Gold columns: ['ID', 'label']
  Pred columns: ['ID', 'label']

📊 Accuracy Results
  Total samples: 182
  Correct predictions: 171
  Incorrect predictions: 11
  Accuracy: 93.9560%

📈 Per-Label Analysis:
  Label 0:   58/  61 correct ( 95.08%)
  Label 1:    3/   3 correct (100.00%)
  Label 2:   11/  14 correct ( 78.57%)
  Label 4:   22/  25 correct ( 88.00%)
  Label 5:    1/   1 correct (100.00%)
  Label 6:   35/  36 correct ( 97.22%)
  Label 8:    5/   5 correct (100.00%)
  Label 10:    2/   2 correct (100.00%)
  Label 12:    0/   1 correct (  0.00%)
  Label 14:   34/  34 correct (100.00%)
